In [2]:
import torch.nn as nn
import torch




In [77]:
class Multihead_attention(nn.Module):
    def __init__(self, d_in, d_out, context_len, num_heads, dropout, qkv_bias):
        super().__init__()

        assert (d_out % num_heads == 0), \
"d_out must be divisible by num_heads"

        self.d_in = d_in
        self.d_out = d_out
        self.head_dim = d_out//num_heads
        self.context_len = context_len
        self.dropout = dropout
        self.num_heads = num_heads
        self.qkv_bias = qkv_bias

        self.W_query = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_keys = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_values = nn.Linear(d_in, d_out, bias = qkv_bias)

        self.out_proj = nn.Linear(d_out, d_out)

        self.dropout = nn.Dropout(dropout)

        self.register_buffer("mask",
                             torch.triu(torch.ones(self.context_len , self.context_len), diagonal= 1))
        

    def forward(self, x):
        Query = self.W_query(x)
        keys = self.W_keys(x)
        values = self.W_values(x)

        num_batches, num_tokens, d_in = x.shape
        print(f"Shape of x: {x.shape}")

        keys = keys.view(num_batches, num_tokens, self.num_heads, self.head_dim)
        Query = Query.view(num_batches, num_tokens, self.num_heads, self.head_dim)
        values = values.view(num_batches, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1, 2)
        Query = Query.transpose(1, 2)
        values = values.transpose(1, 2)

        print(f"Shape of keys: {keys.shape}")
        print(f"Shape of Query: {Query.shape}")
        print(f"Shape of values: {values.shape}")

        attention_scores = Query @ keys.transpose(2, 3)
        print(f"Shape of attention_scores: {attention_scores.shape}")

        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attention_scores.masked_fill_(mask_bool, -torch.inf)

        attention_weights = torch.softmax(attention_scores / (keys.shape[-1] ** 0.5), dim=-1)
        attention_weights = self.dropout(attention_weights)
        print(f"Shape of attention_weights: {attention_weights.shape}")

        # Correct multiplication and reshaping
        context_vec = (attention_weights @ values).transpose(1, 2)
        print(f"Shape of context_vec after attention: {context_vec.shape}")

        # Now, reshape to combine heads and head_dim into d_out
        context_vec = context_vec.contiguous().view(num_batches, num_tokens, self.d_out)
        print(f"Shape of context_vec reshaped to: {context_vec.shape}")

        context_vec = self.out_proj(context_vec)
        print(f"Shape of context_vec after projection: {context_vec.shape}")

        return context_vec

In [78]:
import torch
inputs = torch.tensor(
[[0.43, 0.15, 0.89], # Your (x^1)
[0.55, 0.87, 0.66], # journey (x^2)
[0.57, 0.85, 0.64], # starts (x^3)
[0.22, 0.58, 0.33], # with (x^4)
[0.77, 0.25, 0.10], # one (x^5)
[0.05, 0.80, 0.55]] # step (x^6)
)

print(f"Shape of single inputs is ",{inputs.shape})
batch = torch.stack((inputs, inputs))

print(f"Shape of batch is ",{batch.shape})

Shape of single inputs is  {torch.Size([6, 3])}
Shape of batch is  {torch.Size([2, 6, 3])}


In [83]:
torch.manual_seed(123)
batch_size, context_len, d_in = batch.shape
d_out = 4
num_heads = 2
dropout = 0.0
ma = Multihead_attention(d_in, d_out, context_len, num_heads, dropout, qkv_bias = False)
context_vec = ma(batch)

Shape of x: torch.Size([2, 6, 3])
Shape of keys: torch.Size([2, 2, 6, 2])
Shape of Query: torch.Size([2, 2, 6, 2])
Shape of values: torch.Size([2, 2, 6, 2])
Shape of attention_scores: torch.Size([2, 2, 6, 6])
Shape of attention_weights: torch.Size([2, 2, 6, 6])
Shape of context_vec after attention: torch.Size([2, 6, 2, 2])
Shape of context_vec reshaped to: torch.Size([2, 6, 4])
Shape of context_vec after projection: torch.Size([2, 6, 4])


In [84]:
context_vec

tensor([[[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0147,  0.3259, -0.0734, -0.3721],
         [-0.0116,  0.3138, -0.0708, -0.3624],
         [-0.0117,  0.2973, -0.0698, -0.3543],
         [-0.0132,  0.2990, -0.0689, -0.3490]],

        [[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0147,  0.3259, -0.0734, -0.3721],
         [-0.0116,  0.3138, -0.0708, -0.3624],
         [-0.0117,  0.2973, -0.0698, -0.3543],
         [-0.0132,  0.2990, -0.0689, -0.3490]]], grad_fn=<ViewBackward0>)

In [82]:
batch

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])

In [ ]:
W_q = 